In [13]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.dictionary_based import BOSSEnsembleDilation
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifierDilation

from sklearn.metrics import accuracy_score

from sktime.benchmarking.data import UEADataset, make_datasets
from sktime.benchmarking.evaluation import Evaluator
from sktime.benchmarking.metrics import PairwiseMetric
from sktime.benchmarking.orchestration import Orchestrator
from sktime.benchmarking.results import HDDResults
from sktime.benchmarking.strategies import TSCStrategy
from sktime.benchmarking.tasks import TSCTask

from sktime.series_as_features.model_selection import PresplitFilesCV

from convst.classifiers import R_DST_Ridge
from convst.utils.dataset_utils import load_sktime_dataset_split

from sktime.datasets import load_from_tsfile
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy

from sklearn.preprocessing import LabelEncoder

### Settings ###

In [18]:

# must be an integer > 0
# 1 for no dilation
max_d = 10

# min_d must be an integer <= max_d and >= 1
min_d = 1

window_sizes = [7, 9, 11]

DATA_PATH = "./Univariate_ts"
datasets = make_datasets(
    path=DATA_PATH, dataset_cls=UEADataset, names=["ArrowHead"]
)

clfs = [
    [BOSSEnsemble(), "BOSS"],
    [BOSSEnsembleDilation(max_dilation=max_d, min_dilation=1), "BOSS_Dilation"],
    #TSCStrategy(TimeSeriesForestClassifier(n_estimators=10), name="TimeSeriesForest"),
    #TSCStrategy(ShapeletTransformClassifier(), name="ShapeletTransform"),
    #TSCStrategy(ShapeletTransformClassifierDilation(), name="ShapeletTransformDilation"),
    [R_DST_Ridge(n_shapelets=10_000, n_jobs=1), "RDST"]
]

results = []

### Benchmark ###

In [19]:

for clf in clfs:
    for dataset in datasets:
        X_train, y_train = load_from_tsfile(dataset._train_path)
        X_test, y_test = load_from_tsfile(dataset._test_path)

        X_train = from_nested_to_3d_numpy(X_train)
        X_test = from_nested_to_3d_numpy(X_test)

        #Convert class labels to make sure they are between 0,n_classes
        le = LabelEncoder().fit(y_train)
        y_train = le.transform(y_train)
        y_test = le.transform(y_test)

        fit_time = time.process_time()
        clf[0].fit(X_train, y_train)
        fit_time = np.round(time.process_time() - fit_time, 5)

        predict_time = time.process_time()
        y_pred = clf[0].predict(X_test)
        pred_time = np.round(time.process_time() - predict_time, 5)

        acc = np.round(accuracy_score(y_test, y_pred), 5)

        results.append((clf[1], dataset, acc, fit_time, pred_time))
        pd.DataFrame.from_records(
            results,
            columns=[
                "Classifier",
                "Dataset",
                "Accuracy",
                "Fit-Time",
                "Predict-Time",
            ],
        ).to_csv("results.csv", index=None)
